# Functional linear modelling

This notebook illustrates how to perform a functional linear modelling of actigraphy data with pyActigraphy

Briefly speaking, the aim of such analysis is to find a "smooth" representation (functions) of the data for subsequent analysis.

Data can be converted to a functional form using either

* basis function expansions (ex: Fourier, B-spline)
* kernel functions


Both types are implemented in `pyActigraphy`.

NB: at the moment, the `pyActigraphy` package allows users to model the daily activity profile only. However, more profiles might be added in the future if needed.

The reference book for 'functional data analysis', including information about functional modelling:

Ramsay, J. O., & Silverman, B. W. (2005). Functional Data Analysis. New York, NY: Springer New York. https://doi.org/10.1007/b98888

In the context of actigraphy, a nice example can be found in:

* Wang, J., Xian, H., Licis, A., Deych, E., Ding, J., McLeland, J., … Shannon, W. (2011). Measuring the impact of apnea and obesity on circadian activity patterns using functional linear modeling of actigraphy data. Journal of Circadian Rhythms, 9(1), 11. https://doi.org/10.1186/1740-3391-9-11

## Imports and input data

In [ ]:
import pyActigraphy
from pyActigraphy.analysis import FLM

In [ ]:
import numpy as np

In [ ]:
import plotly.graph_objs as go

In [ ]:
# create objects for layout and traces
layout = go.Layout(autosize=False, width=850, height=600, title="",xaxis=dict(title=""), shapes=[], showlegend=True)

In [ ]:
import os

In [ ]:
# Define the path to your input data
fpath = os.path.join(os.path.dirname(pyActigraphy.__file__),'tests/data/')

Read an example file:

In [ ]:
raw = pyActigraphy.io.read_raw_awd(fpath+'example_01.AWD', start_time='1918-01-24 08:00:00', period='9 days')

In [ ]:
from pyActigraphy.analysis import FLM

## Basis function expansion

The idea behind a basis function expansion is to represent the data $\{x_i\}_{i\in[0,N]}$ as:

$$ x_i = \beta_{1} \phi_1(t_i) + \beta_{2} \phi_2(t_i) +··· + \beta_{n} \phi_n(t_i)$$ 
where $\{\beta_{i}\}_{i=1}^n$ are scalar coefficients and $\{\phi_i(·)\}_{i=1}^n$ are basis functions.
Possible basis functions include Fourier basis and splines.

Such expansion can be performed with a least-square fit

### Using a Fourier basis expansion

In this case, the basis functions are simple (co)sine functions:
$\phi_{0}(t) = 1$, $\phi_{2n}(t) = cos(n\omega t)$ and $\phi_{2n-1}(t) = sin(n\omega t)$, $\omega = \frac{2\pi}{T}$ where T is the period.

For the daily activity profile, T = 24h.

In [ ]:
# Resampling frequency for the daily activity profile
freq='1min'

In [ ]:
# The number of basis functions is max_order*2+1 (1 constant + n cosine functions + n sine functions)
max_order = 9

First, let's define a FLM object with "Fourier" functions as a basis:

In [ ]:
flm = FLM(basis='fourier',sampling_freq=freq,max_order=max_order)

To estimate the scalar coefficients of the basis expansion ("beta" coefficients), use the `fit` function:

In [ ]:
# By setting the "verbose" parameter to True, the result of least-square fit is displayed:
flm.fit(raw,verbose=True)

Now, to reconstruct the signal using its expansion up to the 9th order:

In [ ]:
flm_est = flm.evaluate(raw)

And compare it to the original daily profile:

In [ ]:
daily_avg = raw.average_daily_activity(binarize=False,freq=freq)

In [ ]:
# set x-axis labels and their corresponding data values
labels = ['00:00', '06:00', '12:00', '18:00']
tickvals = ['00:00:00', '06:00:00', '12:00:00', '18:00:00']

layout = go.Layout(
    autosize=False, width=900, height=600, 
    title="Daily profile",
    xaxis=dict(
        title="Time of day (HH:MM)",
        ticktext=labels,
        tickvals=tickvals), 
    yaxis=dict(title="Counts (a.u)"),
    shapes=[], showlegend=True)

In [ ]:
go.Figure(data=[
    go.Scatter(x=daily_avg.index.astype(str),y=daily_avg,name='Raw activity'),
    go.Scatter(x=daily_avg.index.astype(str),y=flm_est,name='Fourier expansion (9th order)')
],layout=layout)

### Using B-splines

B-splines are piecewise polynomial curves. By definition, they ensure the aforementioned "smoothness" of the data representation.

In [ ]:
daily_avg = raw.average_daily_activity(binarize=False,freq="30min")

In order to check how the data are interpolated, let's set the resampling frequency at 30 min.

In [ ]:
flm_spline = FLM(basis='spline',sampling_freq='30min',max_order=3)

Again, like for the Fourier basis, the evaluation of the B-spline representation is performed via the `fit` function:

In [ ]:
flm_spline.fit(raw, verbose=False)

Now, let's evaluate the splines:

In [ ]:
# The "r" parameter represents the ratio between the number of points at which the spline is evaluated and the original number of points.
r = 10

In [ ]:
spline_est = flm_spline.evaluate(raw,r=r)

To visualize the result, one needs to create 2 different X axis as there are "r" times more points in the evaluated spline:

In [ ]:
t = np.linspace(0,daily_avg.index.size,daily_avg.index.size,endpoint=True)

In [ ]:
t_10 = np.linspace(0,daily_avg.index.size,daily_avg.index.size*r,endpoint=True)

In [ ]:
data = [go.Scatter(x=t,y=daily_avg,name='Raw activity'),
        go.Scatter(x=t_10,y=spline_est,name='B-spline')
       ]

In [ ]:
go.Figure(data=data, layout=layout)

By zooming, you can verify the "smoothness" of the interpolation.

## Gaussian kernel smoothing

As alreayd mentioned, another way to obtain a smooth representation of the data is to apply a smoothing function locally. This can be achieved by convoluting the data with a kernel function.

`pyActigraphy` makes available functions to smooth the data using a gaussian kernel.

When using kernel smoothing, the degree of smoothing is controlled by the so-called bandwith parameter. In our case, this corresponds to the "sigma" parameter of the gaussian kernel (i.e its "width").

This parameter must be chosen as a trade-off between bias and variance: a too small bandwith will yield a high signal variability but a too high bandwith will result in a loss of details. 

The `pyActigraphy` implements two usual "rules of thumbs" (Scott and Silverman) to automatically chose the bandwith, which usually provide a good starting point to search for the optimal bandwith.

In [ ]:
help(FLM.smooth)

In [ ]:
daily_avg = raw.average_daily_activity(freq=flm.sampling_freq, binarize=False)

In [ ]:
names = ['Raw activity', 'Scott', 'Silverman', 'Bandwith: 20']

In [ ]:
daily_avg_smoothed = []

In [ ]:
daily_avg_smoothed.append(flm.smooth(raw, method='scott', verbose=True))

In [ ]:
daily_avg_smoothed.append(flm.smooth(raw, method='silverman', verbose=True))

In [ ]:
daily_avg_smoothed.append(flm.smooth(raw, method=20))

In [ ]:
data = [go.Scatter(x=daily_avg.index.astype(str),y=daily_avg_smoothed[n], name=names[n+1]) for n in range(0,len(daily_avg_smoothed))]

In [ ]:
data.insert(0,go.Scatter(x=daily_avg.index.astype(str),y=daily_avg,name=names[0]))

In [ ]:
go.Figure(data=data,layout=layout)

## Group analysis

In order to facilitate the analysis of large datasets, in addition to the 'FML.fit' functions, `pyAcigraphy` implements the 'FLM.fit_reader' functions. Theses functions make use of multiple CPU's if available to speed up the computation.

Instead of using a simple "BaseRaw" object, which reads a single actigraphy recording, it takes as argument a "RawReader" object which allows users to read multiple recordings at once: 

In [ ]:
reader = pyActigraphy.io.read_raw(fpath + 'example_*.AWD', 'AWD', n_jobs=10, prefer='threads', verbose=10)

The 'reader' object contains now the 6 .AWD recordings that are contained in the test directory of the `pyActigraphy` package.

### Using Fourier functions

In [ ]:
# Define a FLM Object that can be (re-)used to fit the data
flm_fourier = FLM(basis='fourier',sampling_freq='10min',max_order=10)

In [ ]:
# Fit all the recordings contained in the "reader":
flm_fourier.fit_reader(reader, verbose_fit=False, n_jobs=2, prefer='threads', verbose_parallel=10)

In [ ]:
y_est_group_fourier = flm_fourier.evaluate_reader(reader,r=10,n_jobs=2, prefer='threads', verbose_parallel=10)

In [ ]:
daily_avg = raw.average_daily_activity(binarize=False,freq='10min')

In [ ]:
go.Figure(data=[go.Scatter(x=daily_avg.index.astype(str),y=v,name=k) for k,v in y_est_group_fourier.items()],layout=layout)

### Using B-splines

This section is similar to the one for the Fourier basis. It revolves around two functions:

* 'fit_reader'
* 'evaluate_reader'

In [ ]:
flm_spline = FLM(basis='spline',sampling_freq='30min',max_order=3)

In [ ]:
# Fit all the rawAWD instances
flm_spline.fit_reader(
    reader, 
    verbose_fit=False, 
    n_jobs=2, 
    prefer='threads', 
    verbose_parallel=10
)

In [ ]:
y_est_group_spline = flm_spline.evaluate_reader(
    reader,
    r=10,
    n_jobs=2,
    prefer='threads',
    verbose_parallel=10
)

In [ ]:
daily_avg = raw.average_daily_activity(binarize=False,freq='3min') # The original profile was sampled at T = 30 min.

In [ ]:
go.Figure(data=[go.Scatter(x=daily_avg.index.astype(str),y=v,name=k) for k,v in y_est_group_spline.items()],layout=layout)

### Gaussian kernel smoothing

Do not forget that the 'reader' is just a container. So, in that case, a simple loop can do the trick:

In [ ]:
# Define a FLM object with the default Fourier basis (they won't be used, so set the max order to 1.)
flm_smooth = FLM(basis='fourier',sampling_freq='1min',max_order=1)

In [ ]:
y_smoothed_group = {
    ireader.display_name: flm_smooth.smooth(
        ireader,
        method='scott',
        verbose=True
    ) for ireader in reader.readers
}

In [ ]:
daily_avg = raw.average_daily_activity(binarize=False,freq='1min')

In [ ]:
go.Figure(data=[go.Scatter(x=daily_avg.index.astype(str),y=v,name=k) for k,v in y_smoothed_group.items()],layout=layout)

Et voilà!